In [56]:
import pandas as pd
import numpy as np

In [57]:
SC = pd.read_csv('/content/Swift Codes.csv')
SC

,Bank,SWIFT code,Check Digits
0,Lloyds Bank,LOYD,C1
1,Barclays Bank,BARC,22
2,Halifax,HLFX,22
3,HSBC,HBUK,4B
4,Natwest,NWBK,2L
5,Santander,ABBY,3E
6,Data Source Bank,DSBX,12


In [58]:
T = pd.read_csv('/content/Transactions.csv')
T

,Transaction ID,Account Number,Sort Code,Bank
0,3888,62230725,95-98-82,Data Source Bank
1,4746,83172326,42-86-38,Barclays Bank
2,5404,34302539,53-28-21,Barclays Bank
3,9013,13350031,93-87-71,Natwest
4,2535,68745993,57-14-32,Barclays Bank
...,...,...,...,...
95,9373,44568613,24-15-12,HSBC
96,2021,72261023,59-17-51,Lloyds Bank
97,6764,10570182,29-72-39,Data Source Bank
98,3286,47326725,21-85-30,Halifax


In the Transactions table, there is a Sort Code field which contains dashes. We need to remove these so just have a 6 digit string

In [59]:
Sort_code_list = list(T['Sort Code'].str.split(pat='-'))
Sort_code_str = pd.DataFrame(["".join(Sort_code_list[i]) for i in range(len(Sort_code_list))],
                             columns=['Sort Code str'])
T = pd.concat([T,Sort_code_str],axis=1)
T

,Transaction ID,Account Number,Sort Code,Bank,Sort Code str
0,3888,62230725,95-98-82,Data Source Bank,959882
1,4746,83172326,42-86-38,Barclays Bank,428638
2,5404,34302539,53-28-21,Barclays Bank,532821
3,9013,13350031,93-87-71,Natwest,938771
4,2535,68745993,57-14-32,Barclays Bank,571432
...,...,...,...,...,...
95,9373,44568613,24-15-12,HSBC,241512
96,2021,72261023,59-17-51,Lloyds Bank,591751
97,6764,10570182,29-72-39,Data Source Bank,297239
98,3286,47326725,21-85-30,Halifax,218530


Use the SWIFT Bank Code lookup table to bring in additional information about the SWIFT code and Check Digits of the receiving bank account

In [60]:
df = T.join(SC.set_index('Bank'),lsuffix='_T',rsuffix='_SC',on='Bank')
df

,Transaction ID,Account Number,Sort Code,Bank,Sort Code str,SWIFT code,Check Digits
0,3888,62230725,95-98-82,Data Source Bank,959882,DSBX,12
1,4746,83172326,42-86-38,Barclays Bank,428638,BARC,22
2,5404,34302539,53-28-21,Barclays Bank,532821,BARC,22
3,9013,13350031,93-87-71,Natwest,938771,NWBK,2L
4,2535,68745993,57-14-32,Barclays Bank,571432,BARC,22
...,...,...,...,...,...,...,...
95,9373,44568613,24-15-12,HSBC,241512,HBUK,4B
96,2021,72261023,59-17-51,Lloyds Bank,591751,LOYD,C1
97,6764,10570182,29-72-39,Data Source Bank,297239,DSBX,12
98,3286,47326725,21-85-30,Halifax,218530,HLFX,22


Add a field for the Country Code

In [61]:
df['Country Code'] = 'GB'
df

,Transaction ID,Account Number,Sort Code,Bank,Sort Code str,SWIFT code,Check Digits,Country Code
0,3888,62230725,95-98-82,Data Source Bank,959882,DSBX,12,GB
1,4746,83172326,42-86-38,Barclays Bank,428638,BARC,22,GB
2,5404,34302539,53-28-21,Barclays Bank,532821,BARC,22,GB
3,9013,13350031,93-87-71,Natwest,938771,NWBK,2L,GB
4,2535,68745993,57-14-32,Barclays Bank,571432,BARC,22,GB
...,...,...,...,...,...,...,...,...
95,9373,44568613,24-15-12,HSBC,241512,HBUK,4B,GB
96,2021,72261023,59-17-51,Lloyds Bank,591751,LOYD,C1,GB
97,6764,10570182,29-72-39,Data Source Bank,297239,DSBX,12,GB
98,3286,47326725,21-85-30,Halifax,218530,HLFX,22,GB


Create the IBANs

In [62]:
IBAN = df['Country Code'] + df['Check Digits'] + df['SWIFT code'] + df['Sort Code str'] + df['Account Number'].astype('str')
IBAN = pd.DataFrame(IBAN,columns=['IBAN'])
output = pd.concat([df['Transaction ID'],IBAN],axis = 1,)
output

,Transaction ID,IBAN
0,3888,GB12DSBX95988262230725
1,4746,GB22BARC42863883172326
2,5404,GB22BARC53282134302539
3,9013,GB2LNWBK93877113350031
4,2535,GB22BARC57143268745993
...,...,...
95,9373,GB4BHBUK24151244568613
96,2021,GBC1LOYD59175172261023
97,6764,GB12DSBX29723910570182
98,3286,GB22HLFX21853047326725


CSV the OUTPUT:

In [63]:
output.to_csv('output.csv',index=False)